# Testing Date Format and Parameter Paths

This notebook tests different date formats and parameter paths to debug the Reform.from_dict issues.

In [1]:
import sys
sys.path.append('../src')

from policyengine_core.reforms import Reform
from policyengine_us import Microsimulation
from reforms import get_option1_reform, eliminate_ss_taxation

c:\Users\dtsax\envs\pe\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Test 1: Basic SS Reform (from reforms.py)

Test if the basic reform from reforms.py works:

In [2]:
print("Testing Option 1 reform (from reform function)...")
try:
    reform1 = get_option1_reform()
    sim1 = Microsimulation(reform=reform1)
    print("✓ SUCCESS: Option 1 reform works")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")

Testing Option 1 reform (from reform function)...
✗ ERROR: AttributeError: 'ParameterNode' object has no attribute 'update'


## Test 2: SS Reform Dict Directly

Test if we can create a reform from the dictionary returned by the helper function:

In [3]:
print("Testing SS reform from dictionary...")
try:
    ss_dict = eliminate_ss_taxation()
    print(f"Dictionary: {ss_dict}")
    reform2 = Reform.from_dict(ss_dict, country_id="us")
    sim2 = Microsimulation(reform=reform2)
    print("✓ SUCCESS: SS reform dict works")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

Testing SS reform from dictionary...
Dictionary: {'gov.irs.social_security.taxability.rate.base': {'2026-01-01:2100-12-31': 0}, 'gov.irs.social_security.taxability.rate.additional': {'2026-01-01:2100-12-31': 0}}
✗ ERROR: AttributeError: 'ParameterNode' object has no attribute 'update'


Traceback (most recent call last):
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\reforms\reform.py", line 150, in apply
    period = period_(period)
             ^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 166, in period
    raise_error(value)
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 139, in raise_error
    raise ValueError(message)
ValueError: Expected a period (eg. '2017', '2017-01', '2017-01-01', ...); got: '2026-01-01:2100-12-31'.
Learn more about legal period formats in OpenFisca:
<https://openfisca.org/doc/coding-the-legislation/35_periods.html#periods-in-simulations>.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dtsax\AppData\Local\Temp\ipykernel_39440\374064692.py", line 6, in <module>
    sim2 = Microsimulation(reform=reform2)
           ^^^^^^^^^^^^^

## Test 3: Labor Supply Parameters Alone

Test if the labor supply elasticity parameters work by themselves:

In [4]:
print("Testing labor supply parameters alone...")
try:
    labor_params = {
        "gov.simulation.labor_supply_responses.elasticities.income": {
            "2024-01-01:2100-12-31": -0.05
        }
    }
    print(f"Labor params: {labor_params}")
    reform3 = Reform.from_dict(labor_params, country_id="us")
    sim3 = Microsimulation(reform=reform3)
    print("✓ SUCCESS: Labor supply parameters work")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

Testing labor supply parameters alone...
Labor params: {'gov.simulation.labor_supply_responses.elasticities.income': {'2024-01-01:2100-12-31': -0.05}}
✗ ERROR: ValueError: Expected a period (eg. '2017', '2017-01', '2017-01-01', ...); got: '2024-01-01:2100-12-31'.
Learn more about legal period formats in OpenFisca:
<https://openfisca.org/doc/coding-the-legislation/35_periods.html#periods-in-simulations>.


Traceback (most recent call last):
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\reforms\reform.py", line 150, in apply
    period = period_(period)
             ^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 166, in period
    raise_error(value)
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 139, in raise_error
    raise ValueError(message)
ValueError: Expected a period (eg. '2017', '2017-01', '2017-01-01', ...); got: '2024-01-01:2100-12-31'.
Learn more about legal period formats in OpenFisca:
<https://openfisca.org/doc/coding-the-legislation/35_periods.html#periods-in-simulations>.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dtsax\AppData\Local\Temp\ipykernel_39440\2791693073.py", line 10, in <module>
    sim3 = Microsimulation(reform=reform3)
           ^^^^^^^^^^^

## Test 4: Combined Parameters

Test if combining SS reform dict with labor parameters works:

In [5]:
print("Testing combined SS reform + labor parameters...")
try:
    ss_dict = eliminate_ss_taxation()
    labor_params = {
        "gov.simulation.labor_supply_responses.elasticities.income": {
            "2024-01-01:2100-12-31": -0.05
        }
    }
    combined = {**ss_dict, **labor_params}
    print(f"Combined dict keys: {list(combined.keys())}")
    reform4 = Reform.from_dict(combined, country_id="us")
    sim4 = Microsimulation(reform=reform4)
    print("✓ SUCCESS: Combined parameters work")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

Testing combined SS reform + labor parameters...
Combined dict keys: ['gov.irs.social_security.taxability.rate.base', 'gov.irs.social_security.taxability.rate.additional', 'gov.simulation.labor_supply_responses.elasticities.income']
✗ ERROR: AttributeError: 'ParameterNode' object has no attribute 'update'


Traceback (most recent call last):
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\reforms\reform.py", line 150, in apply
    period = period_(period)
             ^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 166, in period
    raise_error(value)
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\periods\helpers.py", line 139, in raise_error
    raise ValueError(message)
ValueError: Expected a period (eg. '2017', '2017-01', '2017-01-01', ...); got: '2026-01-01:2100-12-31'.
Learn more about legal period formats in OpenFisca:
<https://openfisca.org/doc/coding-the-legislation/35_periods.html#periods-in-simulations>.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dtsax\AppData\Local\Temp\ipykernel_39440\247347784.py", line 12, in <module>
    sim4 = Microsimulation(reform=reform4)
           ^^^^^^^^^^^^

## Test 5: Different Date Formats

Test different date format options:

In [6]:
print("Testing different date formats...\n")

# Test with just start date (no range)
print("A. Testing single date (no range):")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Single date format works: '2026-01-01'")
except Exception as e:
    print(f"   ✗ Single date failed: {e}")

# Test with year only
print("\nB. Testing year only:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base": {
            "2026": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Year only format works: '2026'")
except Exception as e:
    print(f"   ✗ Year only failed: {e}")

# Test with colon range
print("\nC. Testing colon range:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01:2100-12-31": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Colon range format works: '2026-01-01:2100-12-31'")
except Exception as e:
    print(f"   ✗ Colon range failed: {e}")

# Test with dot range
print("\nD. Testing dot range:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Dot range format works: '2026-01-01.2100-12-31'")
except Exception as e:
    print(f"   ✗ Dot range failed: {e}")

Testing different date formats...

A. Testing single date (no range):
   ✗ Single date failed: 'ParameterNode' object has no attribute 'update'

B. Testing year only:
   ✗ Year only failed: 'ParameterNode' object has no attribute 'update'

C. Testing colon range:
   ✗ Colon range failed: 'ParameterNode' object has no attribute 'update'

D. Testing dot range:
   ✗ Dot range failed: 'ParameterNode' object has no attribute 'update'


## Conclusion

The tests above will help us identify:
1. If basic reforms work
2. If the issue is with extracting dictionaries
3. If labor supply parameters are valid
4. If combining parameters causes issues
5. What date format actually works

In [7]:
from policyengine_us import Microsimulation

sim = Microsimulation()
params = sim.tax_benefit_system.parameters

print("="*60)
print("EXPLORING SOCIAL SECURITY TAXATION PARAMETERS")
print("="*60)

# Check SS taxation parameters
print("\n1. gov.irs.social_security.taxability.rate:")
rate = params.gov.irs.social_security.taxability.rate
print(f"   Type: {type(rate).__name__}")
if hasattr(rate, 'children'):
    print(f"   Children: {list(rate.children.keys())}")

# Check base
print("\n2. gov.irs.social_security.taxability.rate.base:")
if hasattr(rate, 'base'):
    base = rate.base
    print(f"   Type: {type(base).__name__}")
    if hasattr(base, 'children'):
        print(f"   Children: {list(base.children.keys())}")
        # Check each child
        for child_name in list(base.children.keys())[:5]:  # First 5
            child = getattr(base, child_name)
            print(f"     - {child_name}: {type(child).__name__}")

# Check additional
print("\n3. gov.irs.social_security.taxability.rate.additional:")
if hasattr(rate, 'additional'):
    additional = rate.additional
    print(f"   Type: {type(additional).__name__}")
    if hasattr(additional, 'children'):
        print(f"   Children: {list(additional.children.keys())}")

print("\n" + "="*60)
print("EXPLORING LABOR SUPPLY RESPONSE PARAMETERS")
print("="*60)

# Check labor supply parameters
print("\n4. gov.simulation:")
if hasattr(params.gov, 'simulation'):
    simulation = params.gov.simulation
    print(f"   Type: {type(simulation).__name__}")
    if hasattr(simulation, 'children'):
        print(f"   Children: {list(simulation.children.keys())}")
        
    # Check labor_supply_responses
    if hasattr(simulation, 'labor_supply_responses'):
        print("\n5. gov.simulation.labor_supply_responses:")
        lsr = simulation.labor_supply_responses
        print(f"   Type: {type(lsr).__name__}")
        if hasattr(lsr, 'children'):
            print(f"   Children: {list(lsr.children.keys())}")
            
        # Check elasticities
        if hasattr(lsr, 'elasticities'):
            print("\n6. gov.simulation.labor_supply_responses.elasticities:")
            elasticities = lsr.elasticities
            print(f"   Type: {type(elasticities).__name__}")
            if hasattr(elasticities, 'children'):
                print(f"   Children: {list(elasticities.children.keys())}")
                
            # Check income
            if hasattr(elasticities, 'income'):
                print("\n7. gov.simulation.labor_supply_responses.elasticities.income:")
                income = elasticities.income
                print(f"   Type: {type(income).__name__}")
                if hasattr(income, 'children'):
                    print(f"   Children: {list(income.children.keys())}")
else:
    print("   ✗ gov.simulation does not exist!")

print("\n" + "="*60)

EXPLORING SOCIAL SECURITY TAXATION PARAMETERS

1. gov.irs.social_security.taxability.rate:
   Type: ParameterNode
   Children: ['additional', 'base']

2. gov.irs.social_security.taxability.rate.base:
   Type: ParameterNode
   Children: ['benefit_cap', 'excess']
     - benefit_cap: Parameter
     - excess: Parameter

3. gov.irs.social_security.taxability.rate.additional:
   Type: ParameterNode
   Children: ['benefit_cap', 'bracket', 'excess']

EXPLORING LABOR SUPPLY RESPONSE PARAMETERS

4. gov.simulation:
   Type: ParameterNode
   Children: ['branch_to_determine_itemization', 'capital_gains_responses', 'include_health_benefits_in_net_income', 'labor_supply_responses', 'limit_itemized_deductions_to_taxable_income', 'reported_broadband_subsidy', 'reported_snap']

5. gov.simulation.labor_supply_responses:
   Type: ParameterNode
   Children: ['bounds', 'elasticities']

6. gov.simulation.labor_supply_responses.elasticities:
   Type: ParameterNode
   Children: ['income', 'substitution']

7. g

In [8]:
print("Testing FIXED SS taxation elimination...")
try:
    # Use the CORRECT leaf parameter paths with CORRECT date format
    fixed_ss_dict = {
        # Base rate parameters
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026-01-01": 0
        },
        "gov.irs.social_security.taxability.rate.base.excess": {
            "2026-01-01": 0
        },
        # Additional rate parameters
        "gov.irs.social_security.taxability.rate.additional.benefit_cap": {
            "2026-01-01": 0
        },
        "gov.irs.social_security.taxability.rate.additional.bracket": {
            "2026-01-01": 0
        },
        "gov.irs.social_security.taxability.rate.additional.excess": {
            "2026-01-01": 0
        }
    }
    
    reform = Reform.from_dict(fixed_ss_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("✓ SUCCESS: Fixed SS reform works!")
    print(f"  Used {len(fixed_ss_dict)} leaf parameters")
    print(f"  Date format: single date '2026-01-01'")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "-"*60)
print("Testing FIXED labor supply parameter...")
try:
    fixed_labor_dict = {
        "gov.simulation.labor_supply_responses.elasticities.income": {
            "2024-01-01": -0.05  # Single date, not range!
        }
    }
    
    reform = Reform.from_dict(fixed_labor_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("✓ SUCCESS: Fixed labor supply parameter works!")
    print(f"  Date format: single date '2024-01-01'")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")
    import traceback
    traceback.print_exc()

print("\n" + "-"*60)
print("Testing COMBINED fixed parameters...")
try:
    combined_fixed = {**fixed_ss_dict, **fixed_labor_dict}
    
    reform = Reform.from_dict(combined_fixed, country_id="us")
    sim = Microsimulation(reform=reform)
    print("✓ SUCCESS: Combined fixed parameters work!")
    print(f"  Total parameters: {len(combined_fixed)}")
except Exception as e:
    print(f"✗ ERROR: {type(e).__name__}: {e}")

Testing FIXED SS taxation elimination...
✓ SUCCESS: Fixed SS reform works!
  Used 5 leaf parameters
  Date format: single date '2026-01-01'

------------------------------------------------------------
Testing FIXED labor supply parameter...
✓ SUCCESS: Fixed labor supply parameter works!
  Date format: single date '2024-01-01'

------------------------------------------------------------
Testing COMBINED fixed parameters...
✓ SUCCESS: Combined fixed parameters work!
  Total parameters: 6


In [9]:
print("Testing ranges WITH CORRECT leaf parameter paths...\n")

# Test A: Colon range with correct paths
print("A. Testing colon range with correct leaf paths:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026-01-01:2100-12-31": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Colon range works with correct paths!")
except Exception as e:
    print(f"   ✗ Colon range failed: {type(e).__name__}: {str(e)[:100]}")

# Test B: Dot range with correct paths
print("\nB. Testing dot range with correct leaf paths:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026-01-01.2100-12-31": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Dot range works with correct paths!")
except Exception as e:
    print(f"   ✗ Dot range failed: {type(e).__name__}: {str(e)[:100]}")

# Test C: Just year with correct paths
print("\nC. Testing year-only with correct leaf paths:")
try:
    test_dict = {
        "gov.irs.social_security.taxability.rate.base.benefit_cap": {
            "2026": 0
        }
    }
    reform = Reform.from_dict(test_dict, country_id="us")
    sim = Microsimulation(reform=reform)
    print("   ✓ Year-only works with correct paths!")
except Exception as e:
    print(f"   ✗ Year-only failed: {type(e).__name__}: {str(e)[:100]}")

print("\n" + "="*60)
print("CONCLUSION:")
print("="*60)
print("Testing which date format works with the correct parameter paths.")

Testing ranges WITH CORRECT leaf parameter paths...

A. Testing colon range with correct leaf paths:
   ✗ Colon range failed: ValueError: Expected a period (eg. '2017', '2017-01', '2017-01-01', ...); got: '2026-01-01:2100-12-31'.
Learn m

B. Testing dot range with correct leaf paths:
   ✓ Dot range works with correct paths!

C. Testing year-only with correct leaf paths:
   ✓ Year-only works with correct paths!

CONCLUSION:
Testing which date format works with the correct parameter paths.


In [10]:
# Reload the reforms module to get the updated functions
import importlib
import reforms
importlib.reload(reforms)

print("Testing updated reforms from reforms.py...\n")

# Test Option 1 (eliminate SS taxation)
print("1. Testing Option 1 (eliminate SS taxation):")
try:
    reform1 = reforms.get_option1_reform()
    sim1 = Microsimulation(reform=reform1)
    print("   ✓ Option 1 works!")
except Exception as e:
    print(f"   ✗ Option 1 failed: {type(e).__name__}: {str(e)[:80]}")

# Test Option 2 (tax 85% of SS)
print("\n2. Testing Option 2 (tax 85% of SS):")
try:
    reform2 = reforms.get_option2_reform()
    sim2 = Microsimulation(reform=reform2)
    print("   ✓ Option 2 works!")
except Exception as e:
    print(f"   ✗ Option 2 failed: {type(e).__name__}: {str(e)[:80]}")
    import traceback
    traceback.print_exc()

# Test Option 8 (tax 100% of SS)
print("\n3. Testing Option 8 (tax 100% of SS):")
try:
    reform8 = reforms.get_option8_reform()
    sim8 = Microsimulation(reform=reform8)
    print("   ✓ Option 8 works!")
except Exception as e:
    print(f"   ✗ Option 8 failed: {type(e).__name__}: {str(e)[:80]}")

Testing updated reforms from reforms.py...

1. Testing Option 1 (eliminate SS taxation):
   ✓ Option 1 works!

2. Testing Option 2 (tax 85% of SS):
   ✓ Option 2 works!

3. Testing Option 8 (tax 100% of SS):
   ✓ Option 8 works!


## Test 9: Test Fixed Reforms

Test if the updated reforms now work:

## Test 8: Do Ranges Work with Correct Paths?

Test if ranges work when we use the correct leaf parameter paths:

## Test 7: Fixed Parameter Paths and Dates

Now test with the CORRECT parameter paths (leaf nodes) and date format (single date):

## Test 6: Explore Parameter Structure

Let's explore the actual parameter tree to find the correct paths: